# SH3GL2 - Single gene analysis in GP2 data

## Description

Using "individual level data"

### 0. Getting Started

- Loading Python libraries
- Defining functions
- Installing packages

### 1. Copy data from workspace to cloud environment

### 2. Extract SH3GL2

### 3. Annotate SH3GL2 variants

### 4. Extract coding/non-syn variants

### 5. Calculate frequency in cases versus controls

### 6. Calculate frequency (homozygotes) in cases versus controls

### 7. Save out results

## Getting Started

### Loading Python libraries

### Defining functions

In [1]:
# Use the os package to interact with the environment
import os

# Bring in Pandas for Dataframe functionality
import pandas as pd

# Numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

#Import Sys
import sys as sys

In [2]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

### Set paths

In [3]:
# Set up billing project and data path variables
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

## Print the information to check we are in the proper release and billing 
## This will be different for you, the user, depending on the billing project your workspace is on
print('Billing and Workspace')
print(f'Workspace Name: {WORKSPACE_NAME}')
print(f'Billing Project: {BILLING_PROJECT_ID}')
print(f'Workspace Bucket, where you can upload and download data: {WORKSPACE_BUCKET}')
print('')


## GP2 v5.0
## Explicitly define release v5.0 path 
GP2_RELEASE_PATH = 'gs://gp2tier2/release5_11052023'
GP2_CLINICAL_RELEASE_PATH = f'{GP2_RELEASE_PATH}/clinical_data'
GP2_RAW_GENO_PATH = f'{GP2_RELEASE_PATH}/raw_genotypes'
GP2_IMPUTED_GENO_PATH = f'{GP2_RELEASE_PATH}/imputed_genotypes'
print('GP2 v5.0')
print(f'Path to GP2 v5.0 Clinical Data: {GP2_CLINICAL_RELEASE_PATH}')
print(f'Path to GP2 v5.0 Raw Genotype Data: {GP2_RAW_GENO_PATH}')
print(f'Path to GP2 v5.0 Imputed Genotype Data: {GP2_IMPUTED_GENO_PATH}')


## AMP-PD v3.0
## Explicitly define release v3.0 path 
AMP_RELEASE_PATH = 'gs://amp-pd-data/releases/2022_v3release_1115'
AMP_CLINICAL_RELEASE_PATH = f'{AMP_RELEASE_PATH}/clinical'

AMP_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS'
AMP_WGS_RELEASE_PLINK_PATH = os.path.join(AMP_WGS_RELEASE_PATH, 'plink')
AMP_WGS_RELEASE_PLINK_PFILES = os.path.join(AMP_WGS_RELEASE_PLINK_PATH, 'pfiles')

print('AMP-PD v3.0')
print(f'Path to AMP-PD v3.0 Clinical Data: {AMP_CLINICAL_RELEASE_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PFILES}')
print('')

Billing and Workspace
Workspace Name: Endophilin-A
Billing Project: terra-18d8e41c
Workspace Bucket, where you can upload and download data: gs://fc-f7a400c1-827e-48f8-b7b6-90c488a000a4

GP2 v5.0
Path to GP2 v5.0 Clinical Data: gs://gp2tier2/release5_11052023/clinical_data
Path to GP2 v5.0 Raw Genotype Data: gs://gp2tier2/release5_11052023/raw_genotypes
Path to GP2 v5.0 Imputed Genotype Data: gs://gp2tier2/release5_11052023/imputed_genotypes
AMP-PD v3.0
Path to AMP-PD v3.0 Clinical Data: gs://amp-pd-data/releases/2022_v3release_1115/clinical
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink/pfiles



### Install packages

#### Install Plink 1.9 and Plink 2.0

In [4]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink; then
echo "Plink1.9 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink \n    -------"
wget -N http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 
unzip -o plink_linux_x86_64_20190304.zip
echo -e "\n plink downloaded and unzipped in /home/jupyter/tools \n "

fi


if test -e /home/jupyter/tools/plink2; then
echo "Plink2 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink2 \n    -------"
wget -N https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20220603.zip
unzip -o plink2_linux_avx2_20220603.zip
echo -e "\n plink2 downloaded and unzipped in /home/jupyter/tools \n "

fi

Plink1.9 is already installed in /home/jupyter/tools/
Plink2 is already installed in /home/jupyter/tools/


In [5]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Remote restrictions

In [6]:
%%bash

# chmod plink 1.9 
chmod u+x /home/jupyter/tools/plink

In [7]:
%%bash

# chmod plink 2.0
chmod u+x /home/jupyter/tools/plink2

### Install ANNOVAR

In [8]:
%%bash

# Install ANNOVAR:
# https://www.openbioinformatics.org/annovar/annovar_download_form.php

if test -e /home/jupyter/tools/annovar; then

echo "annovar is already installed in /home/jupyter/tools/"
else
echo "annovar is not installed"
cd /home/jupyter/tools/

wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

tar xvfz annovar.latest.tar.gz

fi

annovar is already installed in /home/jupyter/tools/


In [9]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Install ANNOVAR: Download sources of annotation 

In [10]:
%%bash

cd /home/jupyter/tools/annovar/

perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20140902 humandb/
#perl annotate_variation.pl -buildver hg38 -downdb cytoBand humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ensGene humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar exac03 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp147 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp30a humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad211_genome humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ljb26_all humandb/

NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGene.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneMrna.fa.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneVersion.txt.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished downloading annotation files for hg38 build version, with files saved at the 'humandb' directory
NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.idx.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished d

In [11]:
%%bash
ls /home/jupyter/tools/annovar/

annotate_variation.pl
coding_change.pl
convert2annovar.pl
example
humandb
retrieve_seq_from_fasta.pl
table_annovar.pl
variants_reduction.pl


## EUR

## Copy data from GP2 bucket to workspace

In [12]:
# Make a directory
print("Making a working directory")
WORK_DIR = f'/home/jupyter/SH3GL2_GP2/'
shell_do(f'mkdir -p {WORK_DIR}') # f' means f-string - contains expressions to execute the code

Making a working directory


Executing: mkdir -p /home/jupyter/SH3GL2_GP2/


In [13]:
# Check directory where GP2 data is
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_IMPUTED_GENO_PATH}')

Executing: gsutil -u terra-18d8e41c ls gs://gp2tier2/release5_11052023/imputed_genotypes


gs://gp2tier2/release5_11052023/imputed_genotypes/AAC/
gs://gp2tier2/release5_11052023/imputed_genotypes/AFR/
gs://gp2tier2/release5_11052023/imputed_genotypes/AJ/
gs://gp2tier2/release5_11052023/imputed_genotypes/AMR/
gs://gp2tier2/release5_11052023/imputed_genotypes/CAS/
gs://gp2tier2/release5_11052023/imputed_genotypes/EAS/
gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/
gs://gp2tier2/release5_11052023/imputed_genotypes/FIN/
gs://gp2tier2/release5_11052023/imputed_genotypes/MDE/
gs://gp2tier2/release5_11052023/imputed_genotypes/SAS/


In [14]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_IMPUTED_GENO_PATH}/EUR/chr9* {WORK_DIR}')

Executing: gsutil -u terra-18d8e41c -m cp -r gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/chr9* /home/jupyter/SH3GL2_GP2/


Copying gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/chr9_EUR_release5.log...
Copying gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/chr9_EUR_release5.pgen...
Copying gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/chr9_EUR_release5.pvar...
Copying gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/chr9_EUR_release5.psam...
Download already complete for /home/jupyter/SH3GL2_GP2/chr9_EUR_release5.pgen component 0, skipping download but will run integrity checks.
Resuming download for /home/jupyter/SH3GL2_GP2/chr9_EUR_release5.pgen component 1
Download already complete for /home/jupyter/SH3GL2_GP2/chr9_EUR_release5.pgen component 3, skipping download but will run integrity checks.
Download already complete for /home/jupyter/SH3GL2_GP2/chr9_EUR_release5.pgen component 2, skipping download but will run integrity checks.
\ [4/4 files][  6.5 GiB/  6.5 GiB] 100% Done   7.0 MiB/s ETA 00:00:00           
Operation completed over 4 objects/6.5 GiB.                       

### Create a covariate file with GP2 data

In [15]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_CLINICAL_RELEASE_PATH}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_CLINICAL_RELEASE_PATH}/master_key_release5_final.csv {WORK_DIR}')

Executing: gsutil -u terra-18d8e41c ls gs://gp2tier2/release5_11052023/clinical_data


gs://gp2tier2/release5_11052023/clinical_data/master_key_release5_final.csv
gs://gp2tier2/release5_11052023/clinical_data/release5_data_dictionary.csv


Executing: gsutil -u terra-18d8e41c -m cp -r gs://gp2tier2/release5_11052023/clinical_data/master_key_release5_final.csv /home/jupyter/SH3GL2_GP2/


Copying gs://gp2tier2/release5_11052023/clinical_data/master_key_release5_final.csv...
/ [1/1 files][  2.5 MiB/  2.5 MiB] 100% Done                                    
Operation completed over 1 objects/2.5 MiB.                                      


In [16]:
cov = pd.read_csv(f'{WORK_DIR}/master_key_release5_final.csv')
cov.columns

Index(['GP2ID', 'GP2sampleID', 'manifest_id', 'phenotype', 'pheno_for_qc',
       'other_pheno', 'sex_for_qc', 'age', 'age_of_onset', 'age_at_diagnosis',
       'age_at_death', 'race_for_qc', 'family_history', 'region_for_qc',
       'study', 'pruned', 'pruned_reason', 'label', 'related'],
      dtype='object')

In [17]:
cov_reduced = cov[['GP2sampleID','GP2sampleID','sex_for_qc', 'age', 'phenotype']]
cov_reduced.columns = ['FID','IID', 'SEX','AGE','PHENO']
cov_reduced

,FID,IID,SEX,AGE,PHENO
0,IMMUNEPD_000001_s1,IMMUNEPD_000001_s1,1,61.0,PD
1,IMMUNEPD_000002_s1,IMMUNEPD_000002_s1,1,66.0,PD
2,IMMUNEPD_000003_s1,IMMUNEPD_000003_s1,2,55.0,Control
3,IMMUNEPD_000004_s1,IMMUNEPD_000004_s1,1,50.0,Control
4,IMMUNEPD_000005_s1,IMMUNEPD_000005_s1,2,74.0,Control
...,...,...,...,...,...
26723,MDGAP-EBB_000149_s1,MDGAP-EBB_000149_s1,1,NaN,Control
26724,MDGAP-EBB_000150_s1,MDGAP-EBB_000150_s1,2,NaN,Control
26725,MDGAP-EBB_000151_s1,MDGAP-EBB_000151_s1,1,NaN,Control
26726,MDGAP-EBB_000152_s1,MDGAP-EBB_000152_s1,2,NaN,Control


In [18]:
conditions = [
    (cov_reduced['PHENO'] == "PD"),
    (cov_reduced['PHENO'] == "Control")]

In [19]:
choices = [2,1]
cov_reduced['PHENOTYPE'] = np.select(conditions, choices, default=-9).astype(np.int64)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
cov_reduced.reset_index(inplace=True)
cov_reduced.drop(columns=["index"], inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [21]:
cov_reduced.drop(columns=['PHENO'], inplace=True)

In [22]:
sex = cov_reduced[['FID','IID','SEX']]
sex.to_csv(f'{WORK_DIR}/SEX.txt',index=False, sep="\t")
sex

,FID,IID,SEX
0,IMMUNEPD_000001_s1,IMMUNEPD_000001_s1,1
1,IMMUNEPD_000002_s1,IMMUNEPD_000002_s1,1
2,IMMUNEPD_000003_s1,IMMUNEPD_000003_s1,2
3,IMMUNEPD_000004_s1,IMMUNEPD_000004_s1,1
4,IMMUNEPD_000005_s1,IMMUNEPD_000005_s1,2
...,...,...,...
26723,MDGAP-EBB_000149_s1,MDGAP-EBB_000149_s1,1
26724,MDGAP-EBB_000150_s1,MDGAP-EBB_000150_s1,2
26725,MDGAP-EBB_000151_s1,MDGAP-EBB_000151_s1,1
26726,MDGAP-EBB_000152_s1,MDGAP-EBB_000152_s1,2


In [23]:
pheno = cov_reduced[['FID','IID','PHENOTYPE']]
pheno.to_csv(f'{WORK_DIR}/PHENO.txt',index=False, sep="\t")
pheno

,FID,IID,PHENOTYPE
0,IMMUNEPD_000001_s1,IMMUNEPD_000001_s1,2
1,IMMUNEPD_000002_s1,IMMUNEPD_000002_s1,2
2,IMMUNEPD_000003_s1,IMMUNEPD_000003_s1,1
3,IMMUNEPD_000004_s1,IMMUNEPD_000004_s1,1
4,IMMUNEPD_000005_s1,IMMUNEPD_000005_s1,1
...,...,...,...
26723,MDGAP-EBB_000149_s1,MDGAP-EBB_000149_s1,1
26724,MDGAP-EBB_000150_s1,MDGAP-EBB_000150_s1,1
26725,MDGAP-EBB_000151_s1,MDGAP-EBB_000151_s1,1
26726,MDGAP-EBB_000152_s1,MDGAP-EBB_000152_s1,1


## Extract SH3GL2

In [33]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

# SH3GL2: gene positions on hg38 (from https://useast.ensembl.org/index.html)
/home/jupyter/tools/plink2 \
--pfile chr9_EUR_release5 \
--chr 9 \
--from-bp 17579066  \
--to-bp 17797124 \
--make-bed \
--out pheno_SH3GL2_EUR

PLINK v2.00a3.3LM AVX2 Intel (3 Jun 2022)      www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pheno_SH3GL2_EUR.log.
Options in effect:
  --chr 9
  --from-bp 17579066
  --make-bed
  --out pheno_SH3GL2_EUR
  --pfile chr9_EUR_release5
  --to-bp 17797124

Start time: Tue Jul  4 10:17:35 2023
7450 MiB RAM detected; reserving 3725 MiB for main workspace.
Using up to 2 compute threads.
15356 samples (5896 females, 9460 males; 15356 founders) loaded from
chr9_EUR_release5.psam.
994337 variants loaded from chr9_EUR_release5.pvar.
1 binary phenotype loaded (9230 cases, 4966 controls).
2522 variants remaining after main filters.
Writing pheno_SH3GL2_EUR.fam ... done.
Writing pheno_SH3GL2_EUR.bim ... done.
Writing pheno_SH3GL2_EUR.bed ... 0%76%done.
End time: Tue Jul  4 10:17:35 2023


### Visualize plink files bim, fam and bed

In [34]:
%%bash

# Visualize bim file
WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

head pheno_SH3GL2_EUR.bim

9	chr9:17579156:A:G	0	17579156	G	A
9	chr9:17579165:C:T	0	17579165	T	C
9	chr9:17579206:C:T	0	17579206	T	C
9	chr9:17579372:T:C	0	17579372	C	T
9	chr9:17579393:C:A	0	17579393	A	C
9	chr9:17579440:C:G	0	17579440	G	C
9	chr9:17579547:C:A	0	17579547	A	C
9	chr9:17579565:C:T	0	17579565	T	C
9	chr9:17579587:G:T	0	17579587	T	G
9	chr9:17579588:C:T	0	17579588	T	C


In [35]:
%%bash

# Visualize fam file
WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

head pheno_SH3GL2_EUR.fam

0	APGS_000001_s1	0	0	1	2
0	APGS_000002_s1	0	0	1	2
0	APGS_000003_s1	0	0	1	2
0	APGS_000004_s1	0	0	2	2
0	APGS_000005_s1	0	0	1	2
0	APGS_000006_s1	0	0	1	2
0	APGS_000007_s1	0	0	1	2
0	APGS_000008_s1	0	0	1	2
0	APGS_000009_s1	0	0	1	2
0	APGS_000010_s1	0	0	1	2


In [36]:
%%bash

# Visualize bed file
WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

head pheno_SH3GL2_EUR.bed

l�������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������

In [37]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

# Turn binary files into VCF
/home/jupyter/tools/plink \
--bfile pheno_SH3GL2_EUR \
--recode vcf-fid \
--out pheno_SH3GL2_EUR

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pheno_SH3GL2_EUR.log.
Options in effect:
  --bfile pheno_SH3GL2_EUR
  --out pheno_SH3GL2_EUR
  --recode vcf-fid

7450 MB RAM detected; reserving 3725 MB for main workspace.
2522 variants loaded from .bim file.
15356 people (9460 males, 5896 females) loaded from .fam.
14196 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 15356 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%46%47%48%49%50%51%52%53%54%55%56%57%58%59%60%61%62%63%

## Annotate SH3GL2 variants

In [38]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR
export PATH=$PATH:/home/jupyter/tools/rvtests/third/tabix-0.2.6/

### Bgzip and Tabix
bgzip pheno_SH3GL2_EUR.vcf

tabix -f -p vcf pheno_SH3GL2_EUR.vcf.gz
tabix -f -p vcf pheno_SH3GL2_EUR.vcf.gz

In [39]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

### Annotate variants using ANNOVAR: https://annovar.openbioinformatics.org/en/latest/ 
perl /home/jupyter/tools/annovar/table_annovar.pl pheno_SH3GL2_EUR.vcf.gz /home/jupyter/tools/annovar/humandb/ -buildver hg38 \
-out pheno_SH3GL2_EUR.annovar \
-remove -protocol refGene,clinvar_20140902 \
-operation g,f \
--nopolish \
-nastring . \
-vcfinput


NOTICE: Running with system command <convert2annovar.pl  -includeinfo -allsample -withfreq -format vcf4 pheno_SH3GL2_EUR.vcf.gz > pheno_SH3GL2_EUR.annovar.avinput>
NOTICE: Finished reading 2529 lines from VCF file
NOTICE: A total of 2522 locus in VCF file passed QC threshold, representing 2386 SNPs (1521 transitions and 865 transversions) and 136 indels/substitutions
NOTICE: Finished writing allele frequencies based on 36639416 SNP genotypes (23356476 transitions and 13282940 transversions) and 2088416 indels/substitutions for 15356 samples

NOTICE: Running with system command </home/jupyter/tools/annovar/table_annovar.pl pheno_SH3GL2_EUR.annovar.avinput /home/jupyter/tools/annovar/humandb/ -buildver hg38 -outfile pheno_SH3GL2_EUR.annovar -remove -protocol refGene,clinvar_20140902 -operation g,f --nopolish -nastring . -otherinfo>
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=refGene

NOTICE: Running with system command <annot

## Extract coding and non-syn variants

In [40]:
# Visualize multianno file
SH3GL2_EUR = pd.read_csv(f'{WORK_DIR}/pheno_SH3GL2_EUR.annovar.hg38_multianno.txt', sep = '\t')
SH3GL2_EUR

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo15359,Otherinfo15360,Otherinfo15361,Otherinfo15362,Otherinfo15363,Otherinfo15364,Otherinfo15365,Otherinfo15366,Otherinfo15367,Otherinfo15368
0,9,17579156,17579156,A,G,UTR5,SH3GL2,NM_003026:c.-87A>G,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
1,9,17579165,17579165,C,T,UTR5,SH3GL2,NM_003026:c.-78C>T,.,.,...,0/0,0/0,0/0,0/0,0/0,0/1,0/0,0/0,0/0,0/0
2,9,17579206,17579206,C,T,UTR5,SH3GL2,NM_003026:c.-37C>T,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
3,9,17579372,17579372,C,T,intronic,SH3GL2,.,.,.,...,0/0,0/1,0/0,0/0,1/1,0/1,0/1,0/1,0/1,1/1
4,9,17579393,17579393,C,A,intronic,SH3GL2,.,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2517,9,17796670,17796670,G,T,UTR3,SH3GL2,NM_003026:c.*927G>T,.,.,...,1/1,0/1,0/0,0/1,0/1,0/0,0/0,0/1,0/0,0/1
2518,9,17796672,17796672,C,A,UTR3,SH3GL2,NM_003026:c.*929C>A,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
2519,9,17796950,17796950,C,G,UTR3,SH3GL2,NM_003026:c.*1207C>G,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
2520,9,17796975,17796975,C,T,UTR3,SH3GL2,NM_003026:c.*1232C>T,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [41]:
#Number of total varints 
SH3GL2_EUR.count()

Chr               2522
Start             2522
End               2522
Ref               2522
Alt               2522
                  ... 
Otherinfo15364    2522
Otherinfo15365    2522
Otherinfo15366    2522
Otherinfo15367    2522
Otherinfo15368    2522
Length: 15379, dtype: int64

In [42]:
# Filter exonic variants
coding_EUR = SH3GL2_EUR[SH3GL2_EUR['Func.refGene'] == 'exonic']
coding_EUR.count()

Chr               4
Start             4
End               4
Ref               4
Alt               4
                 ..
Otherinfo15364    4
Otherinfo15365    4
Otherinfo15366    4
Otherinfo15367    4
Otherinfo15368    4
Length: 15379, dtype: int64

In [45]:
# Filter exonic and syn vars
coding_synonymous_EUR = SH3GL2_EUR[(SH3GL2_EUR['Func.refGene'] == 'exonic') & (SH3GL2_EUR['ExonicFunc.refGene'] == 'synonymous SNV')]
coding_synonymous_EUR

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo15359,Otherinfo15360,Otherinfo15361,Otherinfo15362,Otherinfo15363,Otherinfo15364,Otherinfo15365,Otherinfo15366,Otherinfo15367,Otherinfo15368
2421,9,17786424,17786424,C,T,exonic,SH3GL2,.,synonymous SNV,SH3GL2:NM_003026:exon4:c.C231T:p.I77I,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
2422,9,17786430,17786430,C,T,exonic,SH3GL2,.,synonymous SNV,SH3GL2:NM_003026:exon4:c.C237T:p.G79G,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
2469,9,17791299,17791299,G,A,exonic,SH3GL2,.,synonymous SNV,SH3GL2:NM_003026:exon7:c.G693A:p.Q231Q,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [46]:
# Filter exonic and non-syn vars
coding_nonsynonymous_EUR = SH3GL2_EUR[(SH3GL2_EUR['Func.refGene'] == 'exonic') & (SH3GL2_EUR['ExonicFunc.refGene'] == 'nonsynonymous SNV')]
coding_nonsynonymous_EUR

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo15359,Otherinfo15360,Otherinfo15361,Otherinfo15362,Otherinfo15363,Otherinfo15364,Otherinfo15365,Otherinfo15366,Otherinfo15367,Otherinfo15368
2489,9,17793465,17793465,G,T,exonic,SH3GL2,.,nonsynonymous SNV,SH3GL2:NM_003026:exon8:c.G827T:p.G276V,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [47]:
print(coding_nonsynonymous_EUR)

      Chr     Start       End Ref Alt Func.refGene Gene.refGene  \
2489    9  17793465  17793465   G   T       exonic       SH3GL2   

     GeneDetail.refGene ExonicFunc.refGene  \
2489                  .  nonsynonymous SNV   

                            AAChange.refGene  ... Otherinfo15359  \
2489  SH3GL2:NM_003026:exon8:c.G827T:p.G276V  ...            0/0   

      Otherinfo15360 Otherinfo15361 Otherinfo15362  Otherinfo15363  \
2489             0/0            0/0            0/0             0/0   

      Otherinfo15364 Otherinfo15365 Otherinfo15366 Otherinfo15367  \
2489             0/0            0/0            0/0            0/0   

     Otherinfo15368  
2489            0/0  

[1 rows x 15379 columns]


In [48]:
coding_nonsynonymous_EUR.to_csv(f'{WORK_DIR}/coding_nonsynonymous.txt', sep = '\t', index = False)

In [49]:
# Filter intronic variants
intronic_EUR = SH3GL2_EUR[SH3GL2_EUR['Func.refGene'] == 'intronic']
intronic_EUR.count()

Chr               2505
Start             2505
End               2505
Ref               2505
Alt               2505
                  ... 
Otherinfo15364    2505
Otherinfo15365    2505
Otherinfo15366    2505
Otherinfo15367    2505
Otherinfo15368    2505
Length: 15379, dtype: int64

In [50]:
# Filter UTR3 variants
UTR3_EUR = SH3GL2_EUR[SH3GL2_EUR['Func.refGene'] == ('UTR3')]
UTR3_EUR.count()

Chr               10
Start             10
End               10
Ref               10
Alt               10
                  ..
Otherinfo15364    10
Otherinfo15365    10
Otherinfo15366    10
Otherinfo15367    10
Otherinfo15368    10
Length: 15379, dtype: int64

In [51]:
# Filter UTR5 variants
UTR5_EUR = SH3GL2_EUR[SH3GL2_EUR['Func.refGene'] == ('UTR5')]
UTR5_EUR.count()

Chr               3
Start             3
End               3
Ref               3
Alt               3
                 ..
Otherinfo15364    3
Otherinfo15365    3
Otherinfo15366    3
Otherinfo15367    3
Otherinfo15368    3
Length: 15379, dtype: int64

In [52]:
# Calculate freq - cases vs controls

In [53]:
reduced_coding_nonsynonymous_EUR = coding_nonsynonymous_EUR[["Chr", "Start", "End", "Gene.refGene"]]
reduced_coding_nonsynonymous_EUR.to_csv(f'{WORK_DIR}/reduced_coding_nonsynonymous_EUR.txt', sep = '\t', index = False, header= False)

In [54]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_SH3GL2_EUR --extract range reduced_coding_nonsynonymous_EUR.txt --assoc --ci 0.95 --adjust --out coding_nonsynonymous_pheno_SH3GL2_EUR --allow-no-sex

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_SH3GL2_EUR.log.
Options in effect:
  --adjust
  --allow-no-sex
  --assoc
  --bfile pheno_SH3GL2_EUR
  --ci 0.95
  --extract range reduced_coding_nonsynonymous_EUR.txt
  --out coding_nonsynonymous_pheno_SH3GL2_EUR

7450 MB RAM detected; reserving 3725 MB for main workspace.
2522 variants loaded from .bim file.
15356 people (9460 males, 5896 females) loaded from .fam.
14196 phenotype values loaded from .fam.
--extract range: 2521 variants excluded.
--extract range: 1 variant remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 15356 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%

In [55]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_SH3GL2_EUR --extract range reduced_coding_nonsynonymous_EUR.txt --make-bed --max-maf 0.05 --out coding_nonsynonymous_EUR_rare_0.05 --allow-no-sex

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_EUR_rare_0.05.log.
Options in effect:
  --allow-no-sex
  --bfile pheno_SH3GL2_EUR
  --extract range reduced_coding_nonsynonymous_EUR.txt
  --make-bed
  --max-maf 0.05
  --out coding_nonsynonymous_EUR_rare_0.05

7450 MB RAM detected; reserving 3725 MB for main workspace.
2522 variants loaded from .bim file.
15356 people (9460 males, 5896 females) loaded from .fam.
14196 phenotype values loaded from .fam.
--extract range: 2521 variants excluded.
--extract range: 1 variant remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 15356 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%

In [56]:
SH3GL2_EUR_freq = pd.read_csv(f'{WORK_DIR}coding_nonsynonymous_pheno_SH3GL2_EUR.assoc', delim_whitespace=True)
SH3GL2_EUR_freq

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR,SE,L95,U95
0,9,chr9:17793465:G:T,17793465,T,0.003955,0.003828,G,0.02706,0.8693,1.034,0.2004,0.6978,1.531


In [57]:
SH3GL2_EUR_freq_adj = pd.read_csv(f'{WORK_DIR}coding_nonsynonymous_pheno_SH3GL2_EUR.assoc.adjusted', delim_whitespace=True)
SH3GL2_EUR_freq_adj

,CHR,SNP,UNADJ,GC,BONF,HOLM,SIDAK_SS,SIDAK_SD,FDR_BH,FDR_BY
0,9,chr9:17793465:G:T,0.8693,0.8693,0.8693,0.8693,0.8693,0.8693,0.8693,0.8693


## Calculate freq of HMZ in cases versus controls

In [58]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_SH3GL2_EUR --extract range reduced_coding_nonsynonymous_EUR.txt --recode A --out coding_nonsynonymous_pheno_SH3GL2_EUR

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_SH3GL2_EUR.log.
Options in effect:
  --bfile pheno_SH3GL2_EUR
  --extract range reduced_coding_nonsynonymous_EUR.txt
  --out coding_nonsynonymous_pheno_SH3GL2_EUR
  --recode A

7450 MB RAM detected; reserving 3725 MB for main workspace.
2522 variants loaded from .bim file.
15356 people (9460 males, 5896 females) loaded from .fam.
14196 phenotype values loaded from .fam.
--extract range: 2521 variants excluded.
--extract range: 1 variant remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 15356 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%3

In [59]:
SH3GL2_EUR_recode = pd.read_csv(f'{WORK_DIR}coding_nonsynonymous_pheno_SH3GL2_EUR.raw', delim_whitespace=True)
SH3GL2_EUR_recode.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:17793465:G:T_T
0,0,APGS_000001_s1,0,0,1,2,0.0
1,0,APGS_000002_s1,0,0,1,2,0.0
2,0,APGS_000003_s1,0,0,1,2,0.0
3,0,APGS_000004_s1,0,0,2,2,0.0
4,0,APGS_000005_s1,0,0,1,2,0.0


In [60]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile  pheno_SH3GL2_EUR --extract range reduced_coding_nonsynonymous_EUR.txt --recode A --out coding_nonsynonymous_pheno_SH3GL2_EUR

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_SH3GL2_EUR.log.
Options in effect:
  --bfile pheno_SH3GL2_EUR
  --extract range reduced_coding_nonsynonymous_EUR.txt
  --out coding_nonsynonymous_pheno_SH3GL2_EUR
  --recode A

7450 MB RAM detected; reserving 3725 MB for main workspace.
2522 variants loaded from .bim file.
15356 people (9460 males, 5896 females) loaded from .fam.
14196 phenotype values loaded from .fam.
--extract range: 2521 variants excluded.
--extract range: 1 variant remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 15356 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%3

In [61]:
# Explore HMZs for chr9:17793465:G:T_T in cases
SH3GL2_hom_cases_EUR = SH3GL2_EUR_recode[(SH3GL2_EUR_recode['chr9:17793465:G:T_T'] == 2) & (SH3GL2_EUR_recode['PHENOTYPE'] == 2)]
SH3GL2_hom_cases_EUR.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:17793465:G:T_T


In [62]:
# Explore HMZs for chr9:17793465:G:T_T in cases   el que hay es: 17793465
SH3GL2_hom_cases_EUR2 = SH3GL2_EUR_recode[(SH3GL2_EUR_recode['chr9:17793465:G:T_T'] == 2) & (SH3GL2_EUR_recode['PHENOTYPE'] == 2)]
SH3GL2_hom_cases_EUR2.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:17793465:G:T_T


In [63]:
SH3GL2_hom_cases_EUR2.shape

(0, 7)

In [64]:
# Explore HMZs for chr9:17793465:G:T_T in controls
SH3GL2_hom_controls_EUR2 = SH3GL2_EUR_recode[(SH3GL2_EUR_recode['chr9:17793465:G:T_T'] == 2) & (SH3GL2_EUR_recode['PHENOTYPE'] == 1)]
SH3GL2_hom_controls_EUR2.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:17793465:G:T_T


In [65]:
SH3GL2_hom_controls_EUR2.shape

(0, 7)

## Save out results..!

In [62]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR} {WORKSPACE_BUCKET}')

Executing: gsutil -mu terra-18d8e41c cp -r /home/jupyter/SH3GL2_GP2/ gs://fc-f7a400c1-827e-48f8-b7b6-90c488a000a4


Copying file:///home/jupyter/SH3GL2_GP2/pheno_SH3GL2_AFR.vcf.gz [Content-Type=text/vcard]...
Copying file:///home/jupyter/SH3GL2_GP2/chr9_AAC_release5.psam [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/SH3GL2_GP2/chr9_AJ_release5.pgen [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/SH3GL2_GP2/pheno_SH3GL2_AFR.bed [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/SH3GL2_GP2/chr9_FIN_release5.log [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
comp